In [ ]:
# In here the user can specify a certain Stoichiometry
# we search the database for all elements contained in this Stoichiometry
# compounds with these elements
# (fleur_initial_cls_wc) we construct reactions equation out of these and 
# calculate an reaction enhalpy for them
# then we have a link to the xps spectra plot
# also allow the user to provide the input from what these reaction should be constructed.